# 霍夫曼編碼

「霍夫曼樹又稱最優二元樹，是一種帶權路徑長度最短的二元樹。所謂樹的帶權路徑長度，就是樹中所有的葉結點的權值乘上其到根結點的路徑長度（若根結點爲0層，葉結點到根結點的路徑長度爲葉結點的層數）。樹的路徑長度是從樹根到每一結點的路徑長度之和，記爲WPL=（W1*L1+W2*L2+W3*L3+...+Wn*Ln），N個權值Wi（i=1,2,...n）構成一棵有N個葉結點的二元樹，相應的葉結點的路徑長度爲Li（i=1,2,...n）。可以證明霍夫曼樹的WPL是最小的。」  ---節錄自維基百科

簡單的說，對文章進行壓縮編碼時，出現頻率越高的單字會越靠近根節點，也就是路徑長度越短，以最大化壓縮比例。

## 以下節錄維基百科中介紹的兩種實現方法:

實現霍夫曼樹的方式有很多種，可以使用優先佇列（Priority Queue）簡單達成這個過程，給與權重較低的符號較高的優先順序（Priority），演算法如下：

⒈把n個終端節點加入優先佇列，則n個節點都有一個優先權Pi，1 ≤ i ≤ n

⒉如果佇列內的節點數>1，則：

⑴從佇列中移除兩個最小的Pi節點，即連續做兩次remove（min（Pi）, Priority_Queue)
⑵產生一個新節點，此節點為⑴之移除節點之父節點，而此節點的權重值為⑴兩節點之權重和
⑶把⑵產生之節點加入優先佇列中
⒊最後在優先佇列裡的點為樹的根節點（root）

而此演算法的時間複雜度（ Time Complexity）為O（n log n）；因為有n個終端節點，所以樹總共有2n-1個節點，使用優先佇列每個迴圈須O（log n）。


此外，有一個更快的方式使時間複雜度降至線性時間（Linear Time）O（n），就是使用兩個佇列（Queue）創件霍夫曼樹。第一個佇列用來儲存n個符號（即n個終端節點）的權重，第二個佇列用來儲存兩兩權重的合（即非終端節點）。此法可保證第二個佇列的前端（Front）權重永遠都是最小值，且方法如下：

⒈把n個終端節點加入第一個佇列（依照權重大小排列，最小在前端）

⒉如果佇列內的節點數>1，則：

⑴從佇列前端移除兩個最低權重的節點
⑵將⑴中移除的兩個節點權重相加合成一個新節點
⑶加入第二個佇列
⒊最後在第一個佇列的節點為根節點

雖然使用此方法比使用優先佇列的時間複雜度還低，但是注意此法的第1項，節點必須依照權重大小加入佇列中，如果節點加入順序不按大小，則需要經過排序，則至少花了O（n log n）的時間複雜度計算。

## 在此採用其方法二並實踐:

### 節點

In [1]:
class _Node():
    
    def __init__(self, Lchild, Rchild):
        # 定義節點屬性，包含左右子節點、權重以及密文。
        # 其中權重即為各單字的出現頻率。
        self.Lchild = Lchild
        self.Rchild = Rchild
        self.weight = Lchild.weight + Rchild.weight
        self.code = b''

### 葉節點

In [2]:
class _Leaf():
    
    def __init__(self, data, weight):
        # 定義葉節點的屬性，權重、單字以及密文。
        self.weight = weight
        self.data = data
        self.code = None

### 霍夫曼樹

In [3]:
class huffman():
    # 用法說明
    '''  Encode:
        >>> huff = huffman(String)
        >>> encoded_string = huff.encoded_text
        >>> decode_key = huff.decode_key
        Decode:
        >>> decode(decode_key, encoded_string)
        '''
    
    # 定義霍夫曼樹的屬性，包含儲存節點、葉節點的串列，但使用上將其當作堆疊。
    # 計算輸入的字串的詞頻，初始化葉節點並根據其權重建構出霍夫曼樹。
    def __init__(self, text):
        self.__leaf = []
        self.__node = []
        self.__word_count(text)
        self.__init_leaf()
        self.__grow()
        self.__encode(text)
    
    # 計算輸入字串的詞頻
    def __word_count(self, text):        
        word_count_dict = dict()
        for i in text:
            if i in word_count_dict:
                word_count_dict[i] += 1
            else:
                word_count_dict[i] = 1                
        self.word_frequency =  sorted(list((word_count_dict.items())), key=lambda x: x[1])
    
    # 依據詞頻建構出葉節點，並將其放入堆疊。
    def __init_leaf(self):
        for i in self.word_frequency:
            leaf = _Leaf(i[0], i[1])
            self.__leaf.append(leaf)
    
    # 依據堆疊中的葉節點，逐步建構出各節點，最後生成霍夫曼樹。
    def __grow(self):
        
#         只有一個葉節點時，其也為根節點。
        if len(self.__leaf) == 1:
            self.__root = self.__leaf[0]
            self.__root.code = b'0'
            
        # 把n個終端節點加入第一個佇列（依照權重大小排列，最小在前端）
        # ⒉如果佇列內的節點數>1，則：
        #     ⑴從佇列前端移除兩個最低權重的節點
        #     ⑵將⑴中移除的兩個節點權重相加合成一個新節點
        #     ⑶加入第二個佇列
        #⒊最後在第一個佇列的節點為根節點
        else:
            while len(self.__leaf) > 0 or len(self.__node) != 1:            
                if self.__node != []:
                    child1 = self.__leaf.pop(0) if len(self.__node) == 1 else self.__node.pop(0)
                    child2 = self.__node.pop(0) if len(self.__leaf) == 0 or self.__node[0].weight <= self.__leaf[0].weight else self.__leaf.pop(0)                
                    node = _Node(child1, child2) if child1.weight <= child2.weight else _Node(child2, child1)                    
                else:                
                    node = _Node(self.__leaf.pop(0), self.__leaf.pop(0)) 
                self.__node.append(node)
            self.__root = self.__node[0]
    
    # 遍歷霍夫曼樹，產生密文與密鑰。
    def __encode(self, text):
        
        encode_key = {}
        self.encoded_text = b''
        
        # 定義遍歷方法，遍歷至葉節點即儲存密文與其相對應的密鑰。
        def generate_key(node):
            if type(node) != _Leaf:        
                node.Lchild.code = node.code+b'0'
                generate_key(node.Lchild)
                node.Rchild.code = node.code+b'1'
                generate_key(node.Rchild)
            else:
                encode_key[node.data.encode()] = node.code
        generate_key(self.__root)
        self.decode_key = dict(zip(encode_key.values(), encode_key.keys()))
        
        #將輸入的字串轉換成密文。
        for i in text:
            self.encoded_text += encode_key[i.encode()]

In [4]:
def decode(decode_key, encoded_text):
    # 依照密鑰將密文解密。
    text = b''
    code = ''
    encoded_text = encoded_text.decode()
    for i in encoded_text:
        code += i
        if code.encode() in decode_key:
            text += decode_key[code.encode()]
            code = ''
    return text.decode()

## Demo

#### 將"to be or not to be"進行霍夫曼編碼:

In [5]:
h = huffman('to be or not to be')

#### 詞頻:

In [6]:
h.word_frequency

[('r', 1), ('n', 1), ('b', 2), ('e', 2), ('t', 3), ('o', 4), (' ', 5)]

#### 密鑰:

In [7]:
h.decode_key

{b'000': b'b',
 b'0010': b'r',
 b'0011': b'n',
 b'01': b'o',
 b'100': b'e',
 b'101': b't',
 b'11': b' '}

#### 密文:

In [8]:
h.encoded_text

b'10101110001001101001011001101101111010111000100'

#### 解密:

In [9]:
decode(h.decode_key, h.encoded_text)

'to be or not to be'

## 參考資料:

維基百科:
https://zh.wikipedia.org/wiki/%E9%9C%8D%E5%A4%AB%E6%9B%BC%E7%BC%96%E7%A0%81#%E6%BC%94%E7%AE%97%E9%81%8E%E7%A8%8B